### ARIMA

##### 3 Components:
- AR (AutoRegression)
    - The forecast is a linear combination of it's previous terms
- I (Integrated)
    - Indicates that the data values have been replaced with the difference between their values and the previous values
    - How many times we have to difference the data so AR and MA can work together
- MA (Moving Average)
    - The regression error is a linear combination of error terms whose values occurred contemporaneously and at various times in the past
    - A model that uses the dependency between an observation and a residual error from a moving average model applied to lagged observations
    - Revall that when we plotted out a moving average with pandas it would 'smooth' out the noise from the time series

- Non-seasonal ARIMA models are generally denoted ARIMA (p, d, q) where these parameters are non-negative integers
- p is the order (number of lags) of the autoregressive model, d is the degree of differencing (the number of times the data have had past values subtracted), and q is the order of the moving-average model

# Choosing ARIMA Orders

- Our main priority here is to try to figure out the orders for the AR and MA components, and if we need to difference our data (the I component)
- Depending on the dataset, it is quite common to only require AR or MA components (you may not need both)
- If the autocorrelation plot shows positive autocorrelation at the first lag (lag-1), then it suggests to use the AR terms in relation to the lag
- If the autocorrelation plot shows negative autocorrelation at the first lag, then it suggests using MA terms
- This will allow you to decide what actual values of p, d, and q to provide your ARIMA model

    p: The number of lag observations included in the model
    d: the number of times that the raw observations are differenced
    q: the size of the moving average window, also called the order of moving average

### Autocorrelation
- Typically a sharp drop in autocorrelation after lag "k" suggests that AR-k model should be used
- If there is a gradual decline, it suggests an MA model

- Identification of an AR model is often best done with the PACF
- Identification of an MA model is often best done with the ACF rather than the PACF
- View the notebook and resource links for more details

- As previously mentioned, it can be very difficult to read these plots, so it is often more effective to perform a grid search across various combinations of p, d, q values

### Pyramid ARIMA
- The pmdarima is a separate library designed to perform grid searches across multiple combinations of p, d, q and P, D, Q
- This is by far the most effective way to get good fitting models
- The pmdarima library utilizes the Akaike information criterion (AIC) as a metric to compare the performance of verious ARIMA based models

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# Load a seasonal dataset
df1 = pd.read_csv(r"C:\Users\olive\Documents\GitHub\Time-Series-Data-Analysis\TSA_COURSE_NOTEBOOKS\Data\airline_passengers.csv",
                 index_col='Month', parse_dates=True)
df1.index.freq = 'MS'

df2 = pd.read_csv(r"C:\Users\olive\Documents\GitHub\Time-Series-Data-Analysis\TSA_COURSE_NOTEBOOKS\Data\DailyTotalFemaleBirths.csv",
                 index_col='Date', parse_dates=True)
df2.index.freq = 'D'

In [2]:
from pmdarima import auto_arima

In [4]:
# help(auto_arima)

In [11]:
stepwise_fit = auto_arima(df2['Births'], start_p=0, start_q=0, max_p=6, max_q=3, seasonal=False, trace=True)

Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 0); AIC=2650.760, BIC=2658.555, Fit time=0.013 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 0, 0, 0); AIC=2565.234, BIC=2576.925, Fit time=0.036 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 0, 0); AIC=2463.584, BIC=2475.275, Fit time=0.063 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 0); AIC=2648.768, BIC=2652.665, Fit time=0.007 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 0, 0, 0); AIC=2460.154, BIC=2475.743, Fit time=0.131 seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(0, 0, 0, 0); AIC=2461.271, BIC=2480.757, Fit time=0.171 seconds


C:\Users\olive\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Fit ARIMA: order=(1, 1, 2) seasonal_order=(0, 0, 0, 0); AIC=2460.644, BIC=2480.130, Fit time=0.369 seconds
Near non-invertible roots for order (1, 1, 2)(0, 0, 0, 0); setting score to inf (at least one inverse root too close to the border of the unit circle: 0.997)
Fit ARIMA: order=(0, 1, 2) seasonal_order=(0, 0, 0, 0); AIC=2460.722, BIC=2476.311, Fit time=0.150 seconds
Fit ARIMA: order=(2, 1, 0) seasonal_order=(0, 0, 0, 0); AIC=2536.154, BIC=2551.742, Fit time=0.095 seconds
Fit ARIMA: order=(2, 1, 2) seasonal_order=(0, 0, 0, 0); AIC=2463.030, BIC=2486.413, Fit time=0.498 seconds
Total fit time: 1.538 seconds


C:\Users\olive\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


You might expect AutoARIMA to produce 18 fits (6x3). However since AIC punishes more complex models it may not actually compute all 18. It might stop at a complex model & decide going further would be overkill (potentially overfit)

In [12]:
stepwise_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Statespace Model Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  365
Model:               SARIMAX(1, 1, 1)   Log Likelihood               -1226.077
Date:                Thu, 02 Jan 2020   AIC                           2460.154
Time:                        13:57:14   BIC                           2475.743
Sample:                             0   HQIC                          2466.350
                                - 365                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0132      0.014      0.975      0.330      -0.013       0.040
ar.L1          0.1299      0.059      2.217      0.027       0.015       0.245
ma.L1         -0.9694      0.016    -62.235      0.000      -1.000      -0.939
sigma2        48.9989      3.432     14.279      0.000      42.273      55.725
===================================================================================
Ljung-Box (Q):                       36.69   Jarque-Bera (JB):                26.17
Prob(Q):                              0.62   Prob(JB):                         0.00
Heteroskedasticity (H):               0.97   Skew:                             0.58
Prob(H) (two-sided):                  0.85   Kurtosis:                         3.62
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [15]:
# 'm' is the number of periods in each season
stepwise_fit2 = auto_arima(df1['Thousands of Passengers'], start_p=0, start_q=0, max_p=4, max_q=4,
                         seasonal=True, trace=True, m=12)

Fit ARIMA: order=(0, 1, 0) seasonal_order=(1, 1, 1, 12); AIC=1034.075, BIC=1045.576, Fit time=0.369 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=1033.479, BIC=1039.229, Fit time=0.012 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=1022.316, BIC=1033.817, Fit time=0.244 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=1022.904, BIC=1034.405, Fit time=0.347 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=1031.508, BIC=1034.383, Fit time=0.012 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=1022.343, BIC=1030.968, Fit time=0.104 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(2, 1, 0, 12); AIC=1021.137, BIC=1035.513, Fit time=0.896 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(2, 1, 1, 12); AIC=1017.165, BIC=1034.416, Fit time=3.539 seconds
Near non-invertible roots for order (1, 1, 0)(2, 1, 1, 12); setting score to inf (at least one inverse root too close to the border of t

C:\Users\olive\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Fit ARIMA: order=(0, 1, 1) seasonal_order=(2, 1, 1, 12); AIC=1015.841, BIC=1033.092, Fit time=2.841 seconds
Near non-invertible roots for order (0, 1, 1)(2, 1, 1, 12); setting score to inf (at least one inverse root too close to the border of the unit circle: 0.998)
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=1022.207, BIC=1036.583, Fit time=0.732 seconds
Fit ARIMA: order=(0, 1, 2) seasonal_order=(2, 1, 0, 12); AIC=1022.996, BIC=1040.247, Fit time=0.942 seconds
Total fit time: 16.195 seconds


In [16]:
stepwise_fit2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  144
Model:             SARIMAX(0, 1, 1)x(2, 1, 1, 12)   Log Likelihood                -501.920
Date:                            Thu, 02 Jan 2020   AIC                           1015.841
Time:                                    13:58:44   BIC                           1033.092
Sample:                                         0   HQIC                          1022.851
                                            - 144                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0003      0.032      0.010      0.992      -0.063       0.064
ma.L1         -0.4253      0.068     -6.229      0.000      -0.559      -0.291
ar.S.L12       0.6670      0.160      4.163      0.000       0.353       0.981
ar.S.L24       0.3315      0.095      3.475      0.001       0.145       0.518
ma.S.L12      -0.9746      1.247     -0.781      0.435      -3.420       1.470
sigma2       110.6448    115.074      0.962      0.336    -114.897     336.186
===================================================================================
Ljung-Box (Q):                       53.03   Jarque-Bera (JB):                 7.47
Prob(Q):                              0.08   Prob(JB):                         0.02
Heteroskedasticity (H):               2.82   Skew:                             0.10
Prob(H) (two-sided):                  0.00   Kurtosis:                         4.15
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""